# 1. Dataset Download and Preparation

In [ ]:
import os
import zipfile
import requests

datasets = {
    'satellite_images': 'https://www.kaggle.com/datasets/arjuntyagi25/satellite-images/download',
}

def download_and_unzip(url, extract_to):
    response = requests.get(url)
    zip_path = 'dataset.zip'
    with open(zip_path, 'wb') as f:
        f.write(response.content)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)

download_and_unzip(datasets['satellite_images'], '/content/satellite_images')


In [ ]:
import os
import cv2
import numpy as np

def preprocess_images(src_dir, dest_dir, target_size=(150, 150)):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for filename in os.listdir(src_dir):
        img_path = os.path.join(src_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        dest_path = os.path.join(dest_dir, filename)
        cv2.imwrite(dest_path, img)

preprocess_images('/content/satellite_images', '/content/preprocessed_images')



# 2. Model Training

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

train_dir = '/content/preprocessed_images_train'
val_dir = '/content/preprocessed_images_test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

model.save('/content/sample_data/your_model.keras')


# 3. Model Testing and Prediction

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from google.colab.patches import cv2_imshow

model = load_model('/content/sample_data/your_model.keras')
class_labels = [ 'beach', 'ice', 'mars', 'moon', 'mountain', 'ocean', 'river']

input_image_path = '/content/India_BMNG.jpg'
img = cv2.imread(input_image_path)
height, width, _ = img.shape

patch_size = (150, 150)
stride = 150

for y in range(0, height - patch_size[1] + 1, stride):
    for x in range(0, width - patch_size[0] + 1, stride):
        patch = img[y:y + patch_size[1], x:x + patch_size[0]]

        patch_img = cv2.resize(patch, (150, 150))
        patch_img = image.img_to_array(patch_img)
        patch_img = np.expand_dims(patch_img, axis=0)
        patch_img = preprocess_input(patch_img)

        predictions = model.predict(patch_img)
        class_idx = np.argmax(predictions[0])
        class_label = class_labels[class_idx]

        if class_label not in ['mars', 'moon']:
            color = (0, 255, 0)
            label = f'{class_label}'
            cv2.rectangle(img, (x, y), (x + patch_size[0], y + patch_size[1]), color, 2)
            cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

output_image_path = '/content/annotated_image.jpg'
cv2.imwrite(output_image_path, img)
cv2_imshow(img)
